# Akeneo API – Data Structures

In this notebook we use the Akeneo Client, implemented in the last notebook. We now try to query all endpoints, that give us data. For this purpose an Akeneo PIM instance was created with the sample data, provided by Akeneo.

In [1]:
import shutil

from IPython.display import display
import pandas as pd

from src import akeneo, config

Preparing a directory to store the results:

In [2]:
csv_dir = config.dir_data / "akeneo-api"
shutil.rmtree(csv_dir, ignore_errors=True)
csv_dir.mkdir(parents=True, exist_ok=True)

In [3]:
def print_and_safe_data(name: str, data: list[dict]):
    df = pd.DataFrame(data)

    print(f"--- {name} --------------------------")
    display(df)
    
    df.to_csv(csv_dir / f"{name}.csv", index=False)

Get all routes:

In [4]:
client = akeneo.create_client_from_env()

In [5]:
routes = client.get_routes()
print_and_safe_data("routes", routes)

--- routes --------------------------


,id,path,method,path_vars
0,pim_api_locale_list,/api/rest/v1/locales,GET,[]
1,pim_api_locale_get,/api/rest/v1/locales/{code},GET,[code]
2,pim_api_channel_list,/api/rest/v1/channels,GET,[]
3,pim_api_channel_get,/api/rest/v1/channels/{code},GET,[code]
4,pim_api_channel_create,/api/rest/v1/channels,POST,[]
...,...,...,...,...
58,pim_api_media_file_create,/api/rest/v1/media-files,POST,[]
59,legacy_pim_api_measure_family_list,/api/rest/v1/measure-families,GET,[]
60,legacy_pim_api_measure_family_get,/api/rest/v1/measure-families/{code},GET,[code]
61,pim_api_measurement_family_get,/api/rest/v1/measurement-families,GET,[]


Filter routes for endpoints that give us data:

In [6]:
routes_df = pd.DataFrame(routes)
routes_df[
    (routes_df["method"] == "GET") &
    (routes_df["path_vars"].isin([[]]) | routes_df["id"].str.endswith("_list"))
].sort_values("path")

,id,path,method,path_vars
34,pim_api_association_type_list,/api/rest/v1/association-types,GET,[]
19,pim_api_attribute_group_list,/api/rest/v1/attribute-groups,GET,[]
9,pim_api_attribute_list,/api/rest/v1/attributes,GET,[]
14,pim_api_attribute_option_list,/api/rest/v1/attributes/{attributeCode}/options,GET,[attributeCode]
44,pim_api_category_list,/api/rest/v1/categories,GET,[]
2,pim_api_channel_list,/api/rest/v1/channels,GET,[]
7,pim_api_currency_list,/api/rest/v1/currencies,GET,[]
24,pim_api_family_list,/api/rest/v1/families,GET,[]
29,pim_api_family_variant_list,/api/rest/v1/families/{familyCode}/variants,GET,[familyCode]
0,pim_api_locale_list,/api/rest/v1/locales,GET,[]


Query all these endpoints, print and safe the result to csv files:

In [7]:
list_routes = routes_df[
    (routes_df["method"] == "GET") & routes_df["path_vars"].isin([[]])
].sort_values("path")

for _, id_, path in list_routes[["id", "path"]].itertuples():
    route_id = str(id_)
    name = str(path).split("/")[-1]

    entries: list[dict] = client.request(route_id)
    print_and_safe_data(name, entries)

    # print the attribute options of the first three simple select attributes
    if route_id == "pim_api_attribute_list":
        num_found = 0
        for attr in entries:
            if attr["type"] == "pim_catalog_simpleselect":
                attr_code = attr["code"]
                attr_opts = client.request(
                    "pim_api_attribute_option_list", {"attributeCode": attr_code}
                )
                print_and_safe_data(f"attribute-options__{attr_code}", attr_opts)
                num_found += 1
            if num_found >= 3:
                break
        continue

    # print the family variants of the first three families
    if route_id == "pim_api_family_list":
        num_found = 0
        for family in entries:
            fam_code = family["code"]
            fam_vars = client.request(
                "pim_api_family_variant_list", {"familyCode": fam_code}
            )
            if fam_vars:
                print_and_safe_data(f"family-variants__{fam_code}", fam_vars)
                num_found += 1
            if num_found >= 3:
                break
        continue

    if route_id in ["pim_api_product_model_list", "pim_api_product_list"]:
        values = [entry["values"] for entry in entries]
        print_and_safe_data(f"{name}__values", values)


--- association-types --------------------------


,code,labels,is_two_way,is_quantified
0,PACK,"{'de_DE': 'Pack', 'en_US': 'Pack', 'fr_FR': 'P...",False,False
1,SUBSTITUTION,"{'de_DE': 'Ersatz', 'en_US': 'Substitution', '...",False,False
2,UPSELL,"{'en_US': 'Upsell', 'fr_FR': 'Vente incitative'}",False,False
3,X_SELL,"{'en_US': 'Cross sell', 'fr_FR': 'Vente croisée'}",False,False


--- attribute-groups --------------------------


,code,sort_order,attributes,labels
0,color,8,"[main_color, secondary_color]","{'en_US': 'Color', 'fr_FR': 'Couleur', 'de_DE'..."
1,design,3,[tshirt_style],"{'en_US': 'Design', 'fr_FR': 'Style', 'de_DE':..."
2,ecommerce,6,"[meta_title, meta_description, keywords]","{'en_US': 'Ecommerce', 'fr_FR': 'Ecommerce', '..."
3,erp,1,"[sku, price, erp_name, ean, supplier]","{'en_US': 'ERP', 'fr_FR': 'ERP', 'de_DE': 'ERP'}"
4,manufacturing,6,"[container_material, tshirt_materials]","{'en_US': 'Manufacturing', 'fr_FR': 'Fabricati..."
5,marketing,0,"[name, description, response_time, release_dat...","{'en_US': 'Marketing', 'fr_FR': 'Marketing', '..."
6,medias,9,"[picture, pdf_description, image, notice, vari...","{'en_US': 'Media', 'fr_FR': 'Media', 'de_DE': ..."
7,other,10,[],"{'en_US': 'Other', 'fr_FR': 'Autre', 'de_DE': ..."
8,product,4,"[composition, wash_temperature, care_instructi...","{'en_US': 'Product', 'fr_FR': 'Produit', 'de_D..."
9,size,8,[clothing_size],"{'en_US': 'Size', 'fr_FR': 'Taille', 'de_DE': ..."


--- attributes --------------------------


...

,supplier,wash_temperature,collection,price,name,erp_name,description,color,variation_name,image,material,brand,meta_title,care_instructions,size,variation_image,composition
0,"[{'locale': None, 'scope': None, 'data': 'zaro'}]","[{'locale': None, 'scope': None, 'data': '800'}]","[{'locale': None, 'scope': None, 'data': ['sum...","[{'locale': None, 'scope': None, 'data': [{'am...","[{'locale': None, 'scope': None, 'data': 'Heri...","[{'locale': 'en_US', 'scope': None, 'data': 'A...","[{'locale': 'en_US', 'scope': 'ecommerce', 'da...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,"[{'locale': None, 'scope': None, 'data': ['sum...",NaN,"[{'locale': None, 'scope': None, 'data': 'Broo...","[{'locale': 'en_US', 'scope': None, 'data': 'B...","[{'locale': 'en_US', 'scope': 'ecommerce', 'da...","[{'locale': None, 'scope': None, 'data': 'blue'}]","[{'locale': 'en_US', 'scope': None, 'data': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"[{'locale': None, 'scope': None, 'data': ['sum...",NaN,"[{'locale': None, 'scope': None, 'data': 'Broo...","[{'locale': 'en_US', 'scope': None, 'data': 'B...","[{'locale': 'en_US', 'scope': 'ecommerce', 'da...","[{'locale': None, 'scope': None, 'data': 'pink'}]","[{'locale': 'en_US', 'scope': None, 'data': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'locale': None, 'scope': None, 'data': 'mong...","[{'locale': None, 'scope': None, 'data': '600'}]","[{'locale': None, 'scope': None, 'data': ['sum...","[{'locale': None, 'scope': None, 'data': [{'am...","[{'locale': None, 'scope': None, 'data': 'Tuxe...","[{'locale': 'en_US', 'scope': None, 'data': 'C...","[{'locale': 'en_US', 'scope': 'ecommerce', 'da...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'locale': None, 'scope': None, 'data': 'abib...",NaN,"[{'locale': None, 'scope': None, 'data': ['sum...","[{'locale': None, 'scope': None, 'data': [{'am...","[{'locale': None, 'scope': None, 'data': 'clim...","[{'locale': 'en_US', 'scope': None, 'data': 'C...",NaN,NaN,NaN,"[{'locale': None, 'scope': None, 'data': 'd/4/...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"[{'locale': None, 'scope': None, 'data': 'zaro'}]","[{'locale': None, 'scope': None, 'data': '400'}]","[{'locale': None, 'scope': None, 'data': ['sum...","[{'locale': None, 'scope': None, 'data': [{'am...","[{'locale': None, 'scope': None, 'data': 'Cott...","[{'locale': 'en_US', 'scope': None, 'data': 'T...","[{'locale': 'en_US', 'scope': 'ecommerce', 'da...","[{'locale': None, 'scope': None, 'data': 'blue'}]","[{'locale': 'en_US', 'scope': None, 'data': 'C...","[{'locale': None, 'scope': None, 'data': '9/5/...","[{'locale': None, 'scope': None, 'data': 'poly...","[{'locale': None, 'scope': None, 'data': 'aken...","[{'locale': 'en_US', 'scope': None, 'data': 'C...","[{'locale': None, 'scope': None, 'data': 'Mach...",NaN,"[{'locale': None, 'scope': None, 'data': 'd/0/...","[{'locale': None, 'scope': None, 'data': '57% ..."
77,"[{'locale': None, 'scope': None, 'data': 'zaro'}]","[{'locale': None, 'scope': None, 'data': '600'}]","[{'locale': None, 'scope': None, 'data': ['sum...","[{'locale': None, 'scope': None, 'data': [{'am...","[{'locale': None, 'scope': None, 'data': 'Slig...","[{'locale': 'en_US', 'scope': None, 'data': 'B...","[{'locale': 'en_US', 'scope': 'ecommerce', 'da...",NaN,NaN,"[{'locale': None, 'scope': None, 'data': '0/9/...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,"[{'locale': None, 'scope': None, 'data': 'zaro'}]","[{'locale': None, 'scope': None, 'data': '400'}]","[{'locale': None, 'scope': None, 'data': ['sum...","[{'locale': None, 'scope': None, 'data': [{'am...","[{'locale': None, 'scope': None, 'data': 'Cott...","[{'locale': 'en_US', 'scope': None, 'data': 'T...","[{'locale': 'en_US', 'scope': 'ecommerce', 'da...","[{'locale': None, 'scope': None, 'data': 'grey'}]","[{'locale': 'en_US', 'scope': None, 'data': 'C...","[{'locale': None, 'scope': None, 'data': '9/5/...","[{'locale': None, 'scope': None, 'data': 'cott...","[{'

--- products --------------------------


,identifier,enabled,family,categories,groups,parent,values,created,updated,associations,quantified_associations
0,1111111171,True,accessories,"[master_accessories_bags, print_accessories, s...",[],None,"{'image': [{'locale': None, 'scope': None, 'da...",2022-03-20T14:06:30+00:00,2022-03-20T14:06:30+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
1,13620748,True,led_tvs,"[led_tvs, samsung, tvs_projectors_sales]",[],None,"{'name': [{'locale': None, 'scope': None, 'dat...",2022-03-20T14:06:30+00:00,2022-03-20T14:06:30+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
2,15554974,True,webcams,"[cameras_sales, logitech, webcams]",[],None,"{'maximum_video_resolution': [{'locale': None,...",2022-03-20T14:06:30+00:00,2022-03-20T14:06:30+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
3,12249740,True,scanners,"[avision, print_scan_sales, scanners]",[],None,"{'picture': [{'locale': None, 'scope': None, '...",2022-03-20T14:06:52+00:00,2022-03-20T14:06:53+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
4,1111111130,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],apollon_yellow,"{'size': [{'locale': None, 'scope': None, 'dat...",2022-03-20T14:06:55+00:00,2022-03-20T14:06:55+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
...,...,...,...,...,...,...,...,...,...,...,...
1234,1111111125,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],apollon_pink,"{'size': [{'locale': None, 'scope': None, 'dat...",2022-03-20T14:06:52+00:00,2022-03-20T14:06:53+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
1235,1111111126,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],apollon_red,"{'size': [{'locale': None, 'scope': None, 'dat...",2022-03-20T14:06:52+00:00,2022-03-20T14:06:53+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
1236,1111111127,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],apollon_red,"{'size': [{'locale': None, 'scope': None, 'dat...",2022-03-20T14:06:52+00:00,2022-03-20T14:06:53+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
1237,1111111128,True,clothing,"[master_men_blazers_deals, supplier_zaro]",[],apollon_yellow,"{'size': [{'locale': None, 'scope': None, 'dat...",2022-03-20T14:06:52+00:00,2022-03-20T14:06:53+00:00,"{'PACK': {'products': [], 'product_models': []...",{}


--- products__values --------------------------


,image,ean,name,weight,display_diagonal,description,release_date,maximum_video_resolution,total_megapixels,picture,...,power_requirements,eu_shoes_size,maximum_frame_rate,image_stabilizer,brand,material,meta_title,composition,care_instructions,main_color
0,"[{'locale': None, 'scope': None, 'data': '8/f/...","[{'locale': None, 'scope': None, 'data': '1234...","[{'locale': None, 'scope': None, 'data': 'Bag'}]","[{'locale': None, 'scope': None, 'data': {'amo...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,"[{'locale': None, 'scope': None, 'data': 'Sams...",NaN,"[{'locale': None, 'scope': None, 'data': {'amo...","[{'locale': 'en_US', 'scope': 'print', 'data':...","[{'locale': None, 'scope': 'ecommerce', 'data'...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"[{'locale': None, 'scope': None, 'data': 'Logi...",NaN,NaN,"[{'locale': 'en_US', 'scope': 'print', 'data':...","[{'locale': None, 'scope': 'ecommerce', 'data'...","[{'locale': None, 'scope': None, 'data': '640_...","[{'locale': None, 'scope': None, 'data': '5'}]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,"[{'locale': None, 'scope': None, 'data': 'Avis...",NaN,NaN,"[{'locale': 'de_DE', 'scope': 'print', 'data':...","[{'locale': None, 'scope': 'ecommerce', 'data'...",NaN,NaN,"[{'locale': None, 'scope': None, 'data': '6/b/...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'locale': None, 'scope': None, 'data': '7/5/...","[{'locale': None, 'scope': None, 'data': '1234...","[{'locale': None, 'scope': None, 'data': 'Long...","[{'locale': None, 'scope': None, 'data': {'amo...",NaN,"[{'locale': 'en_US', 'scope': 'ecommerce', 'da...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,"[{'locale': None, 'scope': None, 'data': '7/5/...","[{'locale': None, 'scope': None, 'data': '1234...","[{'locale': None, 'scope': None, 'data': 'Long...","[{'locale': None, 'scope': None, 'data': {'amo...",NaN,"[{'locale': 'en_US', 'scope': 'ecommerce', 'da...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1235,"[{'locale': None, 'scope': None, 'data': '7/5/...","[{'locale': None, 'scope': None, 'data': '1234...","[{'locale': None, 'scope': None, 'data': 'Long...","[{'locale': None, 'scope': None, 'data': {'amo...",NaN,"[{'locale': 'en_US', 'scope': 'ecommerce', 'da...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1236,"[{'locale': None, 'scope': None, 'data': '7/5/...","[{'locale': None, 'scope': None, 'data': '1234...","[{'locale': None, 'scope': None, 'data': 'Long...","[{'locale': None, 'scope': None, 'data': {'amo...",NaN,"[{'locale': 'en_US', 'scope': 'ecommerce', 'da...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1237,"[{'locale': None, 'scope': None, 'data': '7/5/...","[{'locale': None, 'scope': None, 'data': '1234...","[{'locale': None, 'scope': None, 'data': 'Long...","[{'locale': None, 'scope': None, 'data': {'amo...",NaN,"[{'locale': 'en_US', 'scope': 'ecommerce', 'da...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
